In [10]:
import sys
from logging import CRITICAL, DEBUG, INFO
from time import sleep

import cv2 as cv
import numpy as np

import utils.logger as logger
from camera.video_frame_initializer import initializer
from face_recognizer.face_recognizer import FaceRecognizer
from motion_detector.motion_detector import MotionDetector, MotionDetectorMap
from people_detector.people_detector import PeopleDetector
from utils.bbox_utils import crop_bboxes
from utils.view import view


## PROCESS VIDEO
*Here happens the magick*


In [11]:
def process_video(videos: list[str], motion_detector_map: MotionDetectorMap, people_detector: PeopleDetector,
                  face_recognizer: FaceRecognizer, scale: float = 0.5,
                  early_stop=None, starts_from=0):
    """
    Processes the input video.
    Parameters:
        videos_path: Paths to the video files.
        scale: Scaling factor for resizing frames.
        overlap_threshold: Threshold for merging overlapping detections using IoU.
        area_threshold: Minimum area for detected bounding boxes.
        people_detector: YOLO model to detect people.
        early_stop: stop after n frames
    """
    controller = initializer(videos, timeout=0.1,)
    controller.start_frame_sources()
    # controller.start() # avoid multithread :(
    sleep(1)
    print( controller.has_empty_buffer(), controller.has_alive_sources())
    while not controller.has_empty_buffer() or controller.has_alive_sources():
        frames = controller.fetch_and_get_frames()

        if len(frames) == 0:
            continue

        for video_id, frame in frames:
            # frame = cv.resize(frame, None, fx=scale, fy=scale, interpolation=cv.INTER_LINEAR)
            frame = cv.resize(frame, None, fx=scale, fy=scale, interpolation=cv.INTER_AREA)
            
            motion_detector = motion_detector_map.get_motion_detector(video_id)
            detections, merged_detections, frame = motion_detector(frame, draw=False)

            if not merged_detections:
                continue

            merged_detections = np.array(merged_detections)
            cropped_frames = crop_bboxes(frame, bboxes=merged_detections[:, :-1])  # a-la numpy

            detections = people_detector.detect_on_frames(cropped_frames)
            for detection in detections:
                probs, _, result = detection
                if len(probs) == 0:
                    continue

                detected_person_image = result.orig_img

                # Face detection + recognition

                # Skip too small images
                if detected_person_image.shape[0] < face_recognizer.min_face_size or detected_person_image.shape[
                    1] < face_recognizer.min_face_size:
                    continue
    
                faces = face_recognizer.recognize_faces(detected_person_image)
                annotated_frame = result.plot()
                view(annotated_frame, winname=video_id)
                if len(faces) > 0:
                    print(f"Frame {video_id}: {len(faces)} faces detected.")
                                     
                    for detected_face in faces:
                        if detected_face['label'] is not None:
                            print(
                                f"Detected face: {detected_face['label']} with confidence {detected_face['confidence']}")
                        else:
                            print(f"Detected unrecognized face! 😭")
    else:
        print('No resources available')
    
    cv.destroyAllWindows()

    if sys.platform == 'darwin':
        for _ in range(4):
            cv.waitKey(1)

    cv.destroyAllWindows()



## Main

In [12]:
yolosize = 'n'
yolo11 = PeopleDetector(f"yolo11{yolosize}.pt", verbose=False, )
yolo11.to('cpu')

overlap_threshold = 0.0005
area_threshold = 700
motion_detector = MotionDetectorMap(area_threshold=area_threshold, overlap_threshold=overlap_threshold)

face_recognizer = FaceRecognizer(threshold=0.5)


In [15]:
videos = \
        """./datasets/WiseNET/wisenet_dataset/video_sets/set_1/video1_1.avi
        ./datasets/WiseNET/wisenet_dataset/video_sets/set_1/video1_2.avi
        ./datasets/WiseNET/wisenet_dataset/video_sets/set_1/video1_3.avi
        ./datasets/WiseNET/wisenet_dataset/video_sets/set_1/video1_4.avi
        ./datasets/WiseNET/wisenet_dataset/video_sets/set_1/video1_5.avi""".split('\n')

videos_path = [video.strip() for video in videos]

logger.init_logger(DEBUG)
process_video(videos_path, motion_detector, yolo11, face_recognizer=face_recognizer,
              scale=1)  # , starts_from=3000, early_stop=3500)

18:49:32 [INFO] VideoSource-video1_1.avi: starting video
18:49:32 [INFO] VideoSource-video1_2.avi: starting video
18:49:32 [INFO] VideoSource-video1_3.avi: starting video
18:49:32 [INFO] VideoSource-video1_4.avi: starting video
18:49:32 [INFO] VideoSource-video1_5.avi: starting video
True True
18:49:34 [INFO] FaceRecognizer: No faces detected in input
18:49:43 [INFO] FaceRecognizer: No faces detected in input
18:49:57 [INFO] FaceRecognizer: No faces detected in input
18:49:59 [INFO] FaceRecognizer: No faces detected in input
18:50:01 [INFO] FaceRecognizer: No faces detected in input
18:50:02 [INFO] FaceRecognizer: No faces detected in input
18:50:03 [INFO] FaceRecognizer: No faces detected in input
18:50:05 [INFO] FaceRecognizer: No faces detected in input
18:50:06 [INFO] FaceRecognizer: No faces detected in input
18:50:07 [INFO] FaceRecognizer: No faces detected in input
18:50:07 [INFO] FaceRecognizer: No faces detected in input
18:50:08 [INFO] FaceRecognizer: No faces detected in inp

# People Detection TEST


Define the base functions for people detection. <br/>
It Takes as input an array of frames containing the motion detected by MOG2 above

In [ ]:
# Load a model
model = PeopleDetector("./yolo11x.pt")  # load an official model, or use local path

In [ ]:
def test_yolo11(video_path: str, scale: float = 0.5):
    # Open the video file
    cap = cv.VideoCapture(video_path)

    # Loop through the video frames
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            # Run YOLO inference on the frame
            # frame = cv.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv.INTER_LINEAR)
            results = model(frame)

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            annotated_frame = cv.resize(annotated_frame, None, fx=scale, fy=scale, interpolation=cv.INTER_LINEAR)
            # Display the annotated frame
            cv.imshow("YOLO Inference", annotated_frame)

            # Break the loop if 'q' is pressed
            if cv.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break

    # Release the video capture object and close the display window
    cap.release()
    cv.destroyAllWindows()

## Main test YOLO11